In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from nplinker_constants import nplinker_setup
nplinker_setup(LDA_PATH='/Users/simon/git/lda/code')

In [11]:
from metabolomics import load_spectra
mgf_file = '/srv/data/crusemann/gnps/METABOLOMICS-SNETS-c36f90ba-download_clustered_spectra-main.mgf'
spectra = load_spectra(mgf_file)

FileNotFoundError: [Errno 2] No such file or directory: '/srv/data/crusemann/gnps/METABOLOMICS-SNETS-c36f90ba-download_clustered_spectra-main.mgf'

In [5]:
nodes_file = '/srv/data/crusemann/gnps/0d51c5b6c73b489185a5503d319977ab..out'

In [6]:
from metabolomics import load_metadata
load_metadata(spectra,nodes_file)

In [10]:
for s in spectra:
    if len(s.annotations) > 0:
        print s.annotations



[('Lomaiviticin A', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('Sorbitane Monopalmitate - Polysorbate 40 in-source fragment', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contaminant_Sodium_Formate_Cluster', 'gnps')]
[('MS_Contami

In [11]:
from metabolomics import load_edges
edge_file = '/srv/data/crusemann/gnps/9a93d720f69143bb9f971db39b5d2ba2.pairsinfo'
load_edges(spectra,edge_file)
from metabolomics import make_families
families = make_families(spectra)

In [12]:
import os,glob
from genomics import loadBGC_from_cluster_files
input_files = []
ann_files = []
# this is really messy - change folders to change which folers are included
# we make two lists of files - the clustering ones and the annotation ones
# clusterin gives us the mol families, networking the metadata (bigscape class etc)
# it's also really slow, because of the antismash file matching....which is a mess
# it works for this data but is incredibly non robust
root_path = '/srv/data/crusemann/bigscape/bigscape_corason_crusemann_complete_annotated_mibigs_mix_automode_20180713/network_files/2018-07-13_16-34-11_hybrids_auto_crusemann_bgcs_automode_mix_mibig'
folders = ['NRPS','Others','PKSI','PKS-NRP_Hybrids','PKSother','RiPPs','Saccharides','Terpene']
# ditched 'mix' and 'others'
for folder in folders:
    fam_file = os.path.join(root_path,folder)
    cluster_file = glob.glob(fam_file + os.sep + folder + "_clustering*")
    annotation_files = glob.glob(fam_file + os.sep + "Network_*")
    input_files.append(cluster_file[0])
    ann_files.append(annotation_files[0])
    
gcf_list,bgc_list, strain_list = loadBGC_from_cluster_files(input_files,ann_files,antismash_dir='/srv/data/crusemann/antismash/justin-20181022/',antismash_format = 'flat')

Next cell adds a random object to each spectrum and gcf object, for doing the permutation testing

In [13]:
# initiate the random ones
for g in gcf_list:
    g.add_random(strain_list)
for s in spectra:
    s.add_random(strain_list)

In [ ]:
from scoring import compute_all_scores,metcalf_scoring
m_scores = compute_all_scores(spectra,gcf_list,strain_list,metcalf_scoring,do_random = True)

In [14]:
gcf_with_mibig = []
for gcf in gcf_list:
    if len(gcf.get_mibig_bgcs()) > 0:
        gcf_with_mibig.append(gcf)
print len(gcf_with_mibig),len(gcf_list)

43 1574


In [15]:
spectra_with_lib = []
for s in spectra:
    if len(s.annotations) > 0:
        spectra_with_lib.append(s)
print len(spectra_with_lib)

109


In [18]:
from genomics import load_mibig_map
mibig_map = load_mibig_map()


In [21]:
name_scores = {}
from scoring import name_scoring,compute_all_scores,metcalf_scoring
name_scores = compute_all_scores(spectra_with_lib,gcf_with_mibig,mibig_map,name_scoring,do_random = False)
for s in spectra_with_lib:
    for g in gcf_with_mibig:
        if name_scores[s][g][0] > 0:
            print s,g,name_scores[s][g]
            sc,m = metcalf_scoring(s,g,strain_list)
            print len(g.random_gcf.strain_set),len(s.random_spectrum.strain_set),sc
            for st in m:
                print st.name

Done 0 of 109
Staurosporine Staurosporine M+H
Best:  100
Staurosporine Staurosporine M+H
Done 100 of 109
Spectrum 41296 with 16 peaks, max_ms2_intensity 66689.96875 <genomics.GCF object at 0x7f28dc4d5b90> (100, None, ('Staurosporine', 'Staurosporine M+H'))
34 53 90
CNT088
CNS744
CNS673
CNS296
CNS051
CNY011
CNH718
CNP193
CNS991
CNX814
CNR921
CNY256
CNY237
CNT798
CNY231
CNY694
CNS325
CNH877
CNQ884
CNY280
CNY234
CNS342
CNR107
CNB458
CNY260
CNY282
CNH962
CNT849
CNY281
CNP105
CNT005
Spectrum 41297 with 32 peaks, max_ms2_intensity 7503.780762 <genomics.GCF object at 0x7f28dc4d5b90> (100, None, ('Staurosporine', 'Staurosporine M+H'))
34 18 69
CNY260
CNY282
CNB458
CNS991
CNT849
CNR921
CNR107
CNS296


In [22]:
from genomics_utilities import get_known_cluster_blast
for b in bgc_list:
    get_known_cluster_blast(b)

In [23]:
from scoring import knownclusterblast_scoring
kcb_scores = compute_all_scores(spectra_with_lib,gcf_list,mibig_map,knownclusterblast_scoring,do_random = False)

Done 0 of 109
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Best:  3
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Staurosporine M+H')
Staurosporine Staurosporine M+H
('Staurosporine', 'Sta

Enterocin Enterocin M+H
('Enterocin', 'Enterocin M+H')
Enterocin Enterocin M+H
('Enterocin', 'Enterocin M+H')
Enterocin Enterocin M+H
('Enterocin', 'Enterocin M+H')
Done 100 of 109


In [45]:
for s in spectra_with_lib:
    if ('Enterocin','gnps') in s.annotations:
        print s.spectrum_id
        found = s
ggg = []
for g,score_tuple in kcb_scores[found].items():
    if score_tuple[0] > 0:
        ggg.append(g)
        print metcalf_scoring(s,g,strain_list),score_tuple,len(g.bgc_list),len(s.random_spectrum.strain_set)
        for b in g.bgc_list:
            print b.metadata['knownclusterblast']

36184
(-143, set([])) (95, None, ('Enterocin', 'Enterocin M+H')) 1 25
[('BGC0000220_c1', '95'), ('BGC0000225_c1', '30'), ('BGC0000233_c1', '30'), ('BGC0000222_c1', '30'), ('BGC0000212_c1', '25'), ('BGC0000242_c1', '25'), ('BGC0000194_c1', '15'), ('BGC0000228_c1', '15'), ('BGC0000227_c1', '15'), ('BGC0000195_c1', '20')]
(-144, set([])) (125, None, ('Enterocin', 'Enterocin M+H')) 2 25
[('BGC0000220_c1', '65'), ('BGC0000212_c1', '20'), ('BGC0000194_c1', '15'), ('BGC0000228_c1', '15'), ('BGC0000227_c1', '15'), ('BGC0001409_c1', '15'), ('BGC0000225_c1', '20'), ('BGC0001074_c1', '20'), ('BGC0000254_c1', '20'), ('BGC0000209_c1', '20')]
[('BGC0000220_c1', '60'), ('BGC0000212_c1', '20'), ('BGC0000194_c1', '15'), ('BGC0000228_c1', '15'), ('BGC0000227_c1', '15'), ('BGC0001409_c1', '15'), ('BGC0001074_c1', '20'), ('BGC0000254_c1', '20'), ('BGC0000209_c1', '20'), ('BGC0000208_c1', '20')]


In [37]:
for b in bgc_list:
    if b.name == 'BGC0000220':
        print b

In [38]:
from genomics import load_mibig_library_json
mibig = load_mibig_library_json('/srv/data/mibig/mibig_json_1.4')

Found 1816 files


In [49]:
for f in families:
    if found in f.spectra:
        family_found = f
print family_found
print metcalf_scoring(family_found,ggg[0],strain_list)

Molecular family with 4 spectra
(153, set([<genomics.Strain object at 0x7f28dc9e6f90>]))


In [ ]:
import pylab as plt
import numpy as np
%matplotlib inline
scores =[]
random_scores = []
for s,g in m_scores.items():
    for gg,sc in g.items():
        random_scores.append(sc[1])
        scores.append(sc[0])
plt.hist(random_scores,bins=50)
plt.xlim([100,150])
plt.figure()
plt.hist(scores,bins=50)
plt.xlim([100,150])
plt.ylim([0,1000])
random_scores.sort()
perc99 = random_scores[int(np.floor(0.99*len(random_scores)))]
print perc99
scores = np.array(scores)
n_above = len(np.where(scores>=perc99)[0])
print (1.0*n_above)/len(scores)

In [ ]:
best_spec = None
best_gcf = None
for spectrum in m_scores:
    for gcf in m_scores[spectrum]:
        if m_scores[spectrum][gcf] == best:
            best_spec = spectrum
            best_gcf = gcf

In [ ]:
print best_spec
print best_gcf
for strain in strain_list:
    print strain,best_spec.has_strain(strain),best_gcf.has_strain(strain)

In [ ]:
print best_spec.metadata['LibraryID']
for bgc in best_gcf.bgc_list:
    print bgc,bgc.bigscape_class,bgc.product_prediction

In [ ]:
link_file = 'mibig_gnps_links_q3_loose.csv'
links = []
import csv
with open(link_file,'r') as f:
    reader = csv.reader(f)
    heads = reader.next()
    print heads
    for line in reader:
        links.append(line)

In [ ]:
link_dict = {}
for link in links:
    short_gnps_name = link[3].split()[0]
    link_dict[short_gnps_name] = link[0]    

In [ ]:
spec_lib_dict = {}
for spectrum in spectra:
    if spectrum.spectrum_id == '125':
        print "Hello"
    if spectrum.metadata['LibraryID']:
        lid = spectrum.metadata['LibraryID'].split()[0]
        if lid in link_dict:
            spec_lib_dict[spectrum] = lid

In [ ]:
for spec in spec_lib_dict:
    bgc_name = link_dict[spec_lib_dict[spec]]
    print bgc_name
    bgc = [b for b in bgc_list if b.name.startswith(bgc_name)]
    print bgc

In [ ]:
print [b.name for b in bgc_list if b.name.startswith('BGC')]

In [ ]:
for b in bgc_list:
    if hasattr(b,'metadata'):
        kcb = b.metadata.get('knownclusterblast',None)
        if kcb:
            print kcb
            break

In [ ]:
for s in kcb_scores:
    for g in kcb_scores[s]:
        sc,_,metadata = kcb_scores[s][g]
        if sc > 0:
            a,_ = metcalf_scoring(s,g,strain_list)
            print sc,metadata,a

In [ ]:
for f in families:
    for s in f.spectra:
        if s.spectrum_id == '95':
            found = f

In [ ]:
for s in found.spectra:
    print s.metadata['LibraryID']

In [ ]:
from genomics import load_mibig_library_json
mibig = load_mibig_library_json('/srv/data/mibig/mibig_json_1.4')
mibig_compound_dict = {}
for b in mibig:
    compound = mibig[b]['general_params']['compounds'][0]['compound']
    mibig_compound_dict[compound] = b
print mibig[b].keys()
compounds = []
for s in spectra:
    if len(s.annotations) > 0:
        for compound,_ in s.annotations:
            compounds.append(compound)
compounds = set(compounds)
for c in compounds:
    if 'derep' in c:
        initial = c.split(';')[0].split('_')[0].lower()
        for co in mibig_compound_dict:
            if initial in co:
                print co,initial,c
        